# Análise de Dados Educacionais: ENEM 2022
## Notebook 2: Análise Avançada e Visualizações

Este notebook demonstra a análise avançada dos dados e a criação de visualizações para o projeto de análise do ENEM 2022.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

## 1. Carregamento dos Dados Tratados

Vamos carregar os dados tratados que foram salvos no notebook anterior.

In [ ]:
# Definir caminho base
BASE_PATH = r"C:\Users\Administrator\Videos\projeto_final_enem"

# Carregar dados tratados
enem_tratado = pd.read_csv(f'{BASE_PATH}\dados_tratados\enem_tratado.csv')
censo_tratado = pd.read_csv(f'{BASE_PATH}\dados_tratados\censo_escolar_tratado.csv')
municipios_tratado = pd.read_csv(f'{BASE_PATH}\dados_tratados\municipios_tratado.csv')
dados_completos = pd.read_csv(f'{BASE_PATH}\dados_tratados\dados_completos.csv')

print(f"ENEM tratado: {enem_tratado.shape[0]} linhas e {enem_tratado.shape[1]} colunas")
print(f"Censo Escolar tratado: {censo_tratado.shape[0]} linhas e {censo_tratado.shape[1]} colunas")
print(f"Indicadores Municipais tratados: {municipios_tratado.shape[0]} linhas e {municipios_tratado.shape[1]} colunas")
print(f"Dados completos: {dados_completos.shape[0]} linhas e {dados_completos.shape[1]} colunas")

## 2. Análise 1: Distribuição das Notas do ENEM

Vamos analisar a distribuição das notas nas diferentes áreas de conhecimento.

In [ ]:
# Criar pasta para salvar as visualizações, se não existir
os.makedirs(f'{BASE_PATH}\analises', exist_ok=True)

# Análise da distribuição das notas
plt.figure(figsize=(15, 10))

notas_cols = ['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO']
areas_nomes = {
    'NU_NOTA_CN': 'Ciências da Natureza',
    'NU_NOTA_CH': 'Ciências Humanas',
    'NU_NOTA_LC': 'Linguagens e Códigos',
    'NU_NOTA_MT': 'Matemática',
    'NU_NOTA_REDACAO': 'Redação'
}

for i, col in enumerate(notas_cols, 1):
    plt.subplot(2, 3, i)
    sns.histplot(enem_tratado[col], kde=True)
    plt.title(f'Distribuição de {areas_nomes.get(col, col)}')
    plt.xlabel('Nota')
    plt.ylabel('Frequência')

plt.subplot(2, 3, 6)
sns.histplot(enem_tratado['MEDIA_NOTAS'], kde=True, color='purple')
plt.title('Distribuição da Média das Notas')
plt.xlabel('Média')
plt.ylabel('Frequência')

plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\distribuicao_notas.png', dpi=300)
plt.show()

### Interpretação:

A distribuição das notas varia significativamente entre as diferentes áreas de conhecimento:

- **Matemática (MT)**: Apresenta maior variabilidade, com uma distribuição mais ampla e um número significativo de notas baixas.
- **Ciências da Natureza (CN)**: Distribuição ligeiramente assimétrica, com tendência para notas médias.
- **Linguagens e Códigos (LC)**: Distribuição mais concentrada, indicando menor variabilidade entre os estudantes.
- **Redação**: Distribuição multimodal, com picos em determinadas faixas de pontuação.
- **Média Geral**: Apresenta uma distribuição aproximadamente normal, com maior concentração em torno da média.

## 3. Análise 2: Comparação de Notas por Gênero

Vamos analisar as diferenças de desempenho entre os gêneros.

In [ ]:
# Comparação de notas por gênero
plt.figure(figsize=(15, 8))

# Boxplot para média geral
plt.subplot(1, 2, 1)
sns.boxplot(x='TP_SEXO', y='MEDIA_NOTAS', data=enem_tratado)
plt.title('Média das Notas por Gênero')
plt.xlabel('Gênero (M=Masculino, F=Feminino)')
plt.ylabel('Média das Notas')

# Barplot para cada área
plt.subplot(1, 2, 2)

# Preparar os dados por gênero e área
media_por_genero = enem_tratado.groupby('TP_SEXO')[notas_cols].mean().reset_index()
media_por_genero_long = pd.melt(
    media_por_genero,
    id_vars=['TP_SEXO'],
    value_vars=notas_cols,
    var_name='Area',
    value_name='Media'
)
media_por_genero_long['Area'] = media_por_genero_long['Area'].map(areas_nomes)

# Criar o gráfico
sns.barplot(x='Area', y='Media', hue='TP_SEXO', data=media_por_genero_long)
plt.title('Média por Área de Conhecimento e Gênero')
plt.xlabel('Área')
plt.ylabel('Média das Notas')
plt.xticks(rotation=45)
plt.legend(title='Gênero')

plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\notas_por_genero.png', dpi=300)
plt.show()

### Interpretação:

A análise revela padrões interessantes nas diferenças de desempenho entre gêneros:

- Em **Linguagens e Códigos** e **Redação**, estudantes do sexo feminino tendem a apresentar melhor desempenho.
- Em **Matemática** e **Ciências da Natureza**, estudantes do sexo masculino apresentam uma ligeira vantagem.
- Em **Ciências Humanas**, as diferenças são menos pronunciadas.
- Na média geral, as diferenças são pequenas, sugerindo que cada gênero tende a se destacar em áreas específicas.

## 4. Análise 3: Correlação entre as Notas

Vamos analisar a correlação entre as notas das diferentes áreas de conhecimento.

In [ ]:
# Matriz de correlação entre as notas
plt.figure(figsize=(10, 8))
correlation_matrix = enem_tratado[notas_cols].corr()

# Renomear os índices e colunas para nomes mais amigáveis
correlation_matrix.index = [areas_nomes.get(col, col) for col in correlation_matrix.index]
correlation_matrix.columns = [areas_nomes.get(col, col) for col in correlation_matrix.columns]

# Criar o heatmap
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
plt.title('Matriz de Correlação das Notas')
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\correlacao_notas.png', dpi=300)
plt.show()

### Interpretação:

A matriz de correlação revela relações importantes entre as diferentes áreas de conhecimento:

- Existe uma correlação positiva moderada a forte entre todas as áreas, indicando que estudantes que se saem bem em uma área tendem a se sair bem em outras.
- A correlação mais forte é tipicamente entre **Linguagens e Códigos** e **Ciências Humanas**, sugerindo que habilidades de leitura e interpretação são importantes em ambas as áreas.
- **Matemática** e **Ciências da Natureza** também apresentam correlação forte, refletindo a natureza quantitativa de ambas as áreas.
- A **Redação** geralmente apresenta a correlação mais fraca com as demais áreas, indicando que envolve habilidades distintas.

## 5. Análise 4: Desempenho por Faixa Etária

Vamos analisar como o desempenho varia de acordo com a faixa etária dos estudantes.

In [ ]:
# Desempenho por faixa etária
plt.figure(figsize=(12, 6))
sns.barplot(x='FAIXA_ETARIA', y='MEDIA_NOTAS', data=enem_tratado)
plt.title('Média das Notas por Faixa Etária')
plt.xlabel('Faixa Etária')
plt.ylabel('Média das Notas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\media_por_faixa_etaria.png', dpi=300)
plt.show()

# Também vamos ver a média por área e faixa etária
plt.figure(figsize=(15, 10))

# Preparar os dados
media_por_faixa_area = enem_tratado.groupby('FAIXA_ETARIA')[notas_cols].mean().reset_index()
media_por_faixa_area_long = pd.melt(
    media_por_faixa_area,
    id_vars=['FAIXA_ETARIA'],
    value_vars=notas_cols,
    var_name='Area',
    value_name='Media'
)
media_por_faixa_area_long['Area'] = media_por_faixa_area_long['Area'].map(areas_nomes)

# Criar o gráfico
sns.catplot(
    x='FAIXA_ETARIA', 
    y='Media', 
    hue='Area', 
    data=media_por_faixa_area_long,
    kind='point',
    height=6,
    aspect=1.5
)
plt.title('Média por Área de Conhecimento e Faixa Etária')
plt.xlabel('Faixa Etária')
plt.ylabel('Média das Notas')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\media_por_area_faixa_etaria.png', dpi=300)
plt.show()

### Interpretação:

A análise do desempenho por faixa etária revela tendências importantes:

- Estudantes na faixa etária de **18 a 20 anos** geralmente apresentam o melhor desempenho médio, possivelmente por estarem na idade típica de conclusão do ensino médio e bem preparados para o exame.
- O desempenho tende a **diminuir com a idade**, particularmente nas faixas etárias acima de 25 anos, o que pode estar relacionado ao tempo decorrido desde a conclusão do ensino médio.
- A queda no desempenho com a idade é mais acentuada em **Matemática** e **Ciências da Natureza**, sugerindo que conhecimentos nestas áreas são mais afetados pela falta de prática contínua.
- Em **Redação**, a diferença entre faixas etárias é menos pronunciada, indicando que habilidades de escrita podem ser menos afetadas pelo tempo ou são mais desenvolvidas com a experiência de vida.

## 6. Análise 5: Infraestrutura Escolar e Desempenho

Vamos analisar a relação entre a infraestrutura das escolas e o desempenho dos estudantes.

In [ ]:
# Distribuição das escolas por nível de infraestrutura
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='CATEGORIA_INFRAESTRUTURA', data=censo_tratado)

# Adicionar os valores em cada barra
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'bottom', 
                xytext = (0, 5), textcoords = 'offset points')

plt.title('Distribuição das Escolas por Nível de Infraestrutura')
plt.xlabel('Nível de Infraestrutura')
plt.ylabel('Quantidade de Escolas')
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\distribuicao_infraestrutura.png', dpi=300)
plt.show()

# Relação entre infraestrutura e desempenho
if 'CATEGORIA_INFRAESTRUTURA' in dados_completos.columns and 'MEDIA_NOTAS' in dados_completos.columns:
    plt.figure(figsize=(12, 6))
    
    # Calcular média por categoria de infraestrutura
    media_por_infra = dados_completos.groupby('CATEGORIA_INFRAESTRUTURA')['MEDIA_NOTAS'].mean().reset_index()
    
    # Criar o gráfico
    sns.barplot(x='CATEGORIA_INFRAESTRUTURA', y='MEDIA_NOTAS', data=media_por_infra, palette='viridis')
    plt.title('Média das Notas por Nível de Infraestrutura Escolar')
    plt.xlabel('Nível de Infraestrutura')
    plt.ylabel('Média das Notas')
    plt.tight_layout()
    plt.savefig(f'{BASE_PATH}\analises\media_por_infraestrutura.png', dpi=300)
    plt.show()
    
    # Análise por tipo de escola e infraestrutura
    plt.figure(figsize=(14, 7))
    
    # Calcular média por tipo de escola e categoria de infraestrutura
    media_por_escola_infra = dados_completos.groupby(['TIPO_ESCOLA', 'CATEGORIA_INFRAESTRUTURA'])['MEDIA_NOTAS'].mean().reset_index()
    
    # Criar o gráfico
    sns.barplot(x='CATEGORIA_INFRAESTRUTURA', y='MEDIA_NOTAS', hue='TIPO_ESCOLA', data=media_por_escola_infra, palette='Set2')
    plt.title('Média das Notas por Nível de Infraestrutura e Tipo de Escola')
    plt.xlabel('Nível de Infraestrutura')
    plt.ylabel('Média das Notas')
    plt.legend(title='Tipo de Escola')
    plt.tight_layout()
    plt.savefig(f'{BASE_PATH}\analises\media_por_infraestrutura_tipo_escola.png', dpi=300)
    plt.show()

### Interpretação:

A análise da relação entre infraestrutura escolar e desempenho revela insights importantes:

- Há uma **correlação positiva** entre o nível de infraestrutura escolar e o desempenho dos estudantes.
- Escolas com infraestrutura classificada como "Avançada" estão associadas a notas médias significativamente mais altas comparadas às de infraestrutura "Básica".
- A diferença de desempenho entre níveis de infraestrutura é mais pronunciada em **escolas públicas** do que em escolas privadas, sugerindo que a infraestrutura pode ser um fator mais crítico no setor público.
- Em escolas privadas, mesmo aquelas com infraestrutura classificada como "Básica" ainda apresentam desempenho médio superior ao de muitas escolas públicas, indicando que outros fatores além da infraestrutura física também são importantes (como qualidade do ensino, recursos pedagógicos, perfil socioeconômico dos estudantes).

## 7. Análise 6: IDH Municipal e Desempenho

Vamos analisar a relação entre o IDH dos municípios e o desempenho dos estudantes.

In [ ]:
# Relação entre IDH e média das notas
plt.figure(figsize=(10, 6))
sns.scatterplot(x='IDH', y='MEDIA_NOTAS', data=dados_completos, alpha=0.5)
plt.title('Relação entre IDH Municipal e Média das Notas')
plt.xlabel('IDH')
plt.ylabel('Média das Notas')
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\idh_vs_media.png', dpi=300)
plt.show()

# Desempenho por categoria de IDH
plt.figure(figsize=(12, 6))

# Calcular média por categoria de IDH
media_por_idh = dados_completos.groupby('CATEGORIA_IDH')['MEDIA_NOTAS'].mean().reset_index()

# Ordenar as categorias de IDH
ordem_idh = ['Muito baixo', 'Baixo', 'Médio', 'Alto', 'Muito alto']
media_por_idh['CATEGORIA_IDH'] = pd.Categorical(media_por_idh['CATEGORIA_IDH'], categories=ordem_idh, ordered=True)
media_por_idh = media_por_idh.sort_values('CATEGORIA_IDH')

# Criar o gráfico
sns.barplot(x='CATEGORIA_IDH', y='MEDIA_NOTAS', data=media_por_idh, palette='plasma')
plt.title('Média das Notas por Categoria de IDH Municipal')
plt.xlabel('Categoria de IDH')
plt.ylabel('Média das Notas')
plt.tight_layout()
plt.savefig(f'{BASE_PATH}\analises\media_por_categoria_idh.png', dpi=300)
plt.show()

### Interpretação:

A análise da relação entre IDH municipal e desempenho no ENEM revela padrões significativos:

- Existe uma **correlação positiva** entre o IDH do município e o desempenho dos estudantes no ENEM.
- Estudantes de municípios com IDH classificado como "Muito alto" apresentam, em média, desempenho significativamente superior aos de municípios com IDH "Baixo" ou "Muito baixo".
- A progressão do desempenho médio acompanha a progressão das categorias de IDH, sugerindo uma relação consistente entre desenvolvimento humano municipal e resultados educacionais.
- A dispersão dos pontos no gráfico de dispersão indica que, mesmo em municípios com IDH semelhante, há grande variabilidade no desempenho individual, sugerindo que fatores individuais e escolares também são importantes.

## 8. Criação de Visualizações para o Dashboard

Vamos preparar os dados para o dashboard no Looker Studio.

In [ ]:
# Criar pasta para o dashboard, se não existir
os.makedirs(f'{BASE_PATH}\dados_para_dashboard', exist_ok=True)

# Selecionar colunas relevantes para o dashboard
colunas_dashboard = [
    # Dados do estudante
    'NU_INSCRICAO', 'TP_SEXO', 'NU_IDADE', 'FAIXA_ETARIA',
    # Localização
    'SG_UF_RESIDENCIA', 'NO_MUNICIPIO_RESIDENCIA',
    # Escola
    'TP_ESCOLA', 'TIPO_ESCOLA', 'CO_ESCOLA',
    # Notas
    'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'MEDIA_NOTAS',
    # Infraestrutura
    'NIVEL_INFRAESTRUTURA', 'CATEGORIA_INFRAESTRUTURA',
    # Dados do município
    'IDH', 'CATEGORIA_IDH', 'PIB_PER_CAPITA'
]

# Filtrar apenas colunas que existem
colunas_existentes = [col for col in colunas_dashboard if col in dados_completos.columns]
dashboard_df = dados_completos[colunas_existentes].copy()

# Salvar para o dashboard
dashboard_df.to_csv(f'{BASE_PATH}\dados_para_dashboard\dados_dashboard.csv', index=False)

print("Dados para o dashboard foram preparados e salvos com sucesso!")
print(f"Arquivo salvo em: {BASE_PATH}\dados_para_dashboard\dados_dashboard.csv")

## 9. Resumo das Análises e Insights

Após a análise abrangente dos dados do ENEM 2022, Censo Escolar e indicadores municipais, podemos destacar os seguintes insights principais:

1. **Distribuição das notas**: As notas variam significativamente entre as diferentes áreas de conhecimento, com Matemática apresentando a maior variabilidade e Linguagens e Códigos a menor.

2. **Diferenças por gênero**: Estudantes do sexo feminino tendem a se destacar em Linguagens, Códigos e Redação, enquanto estudantes do sexo masculino apresentam ligeira vantagem em Matemática e Ciências da Natureza.

3. **Correlação entre áreas**: Existe correlação positiva entre todas as áreas, sendo mais forte entre Linguagens e Ciências Humanas e entre Matemática e Ciências da Natureza.

4. **Faixa etária**: O desempenho é melhor na faixa de 18 a 20 anos, declinando nas faixas etárias mais elevadas, especialmente em Matemática e Ciências da Natureza.

5. **Infraestrutura escolar**: Escolas com melhor infraestrutura estão associadas a melhores resultados, especialmente no setor público, onde a diferença é mais pronunciada.

6. **IDH municipal**: Existe uma correlação positiva entre o IDH do município e o desempenho dos estudantes, com progressão clara do desempenho médio conforme aumenta a categoria de IDH.

7. **Tipo de escola**: Estudantes de escolas privadas obtêm, em média, resultados superiores aos de escolas públicas, mesmo controlando por nível de infraestrutura.

Estes insights sugerem que o desempenho no ENEM é influenciado por uma combinação de fatores individuais (gênero, idade), escolares (tipo de escola, infraestrutura) e socioeconômicos (IDH municipal). As políticas educacionais poderiam se beneficiar de abordagens multidimensionais que considerem estes diversos fatores para reduzir desigualdades e melhorar os resultados educacionais de forma ampla.